In [9]:
# Load packages
import requests 
import pandas as pd 
import time

from bs4 import BeautifulSoup 
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By

In [11]:
def get_current_url(url, job_title, location): 
    # service = Service(executable_path="selenium-webdriver/safari")
    # driver = webdriver.Firefox(service=service)
    service = Service(executable_path="/chromedriver")
    driver = webdriver.Chrome(service=service)
    driver.get(url)
    time.sleep(3)
    driver.find_element("xpath", '//*[@id="searchKeywordsField"]').send_keys(job_title)
    time.sleep(3)
    driver.find_element("xpath", '//*[@id="locationAutoSuggest"]').send_keys(location)
    time.sleep(3)
    driver.find_element("xpath", '/html/body/div[1]/div/div/div[4]/div/div[1]/div/div/div/div/div[2]/div/form/div/div/div[2]/div[4]/button').click()
    time.sleep(3)
    # try: 
    #     driver.find_element("xpath", '//*[@id="jobsearch"]/button').click()
    # except: 
    #     driver.find_element("xpath", '//*[@id="whatWhereFormId"]/div[3]/button').click()
    current_url = driver.current_url
    return current_url

current_url = get_current_url("https://www.jobstreet.com.sg/", "Data Scientist", "Singapore")
print(current_url)

https://www.jobstreet.com.sg/Data-Scientist-jobs/in-Singapore


In [12]:
resp = requests.get(current_url)
content = BeautifulSoup(resp.content, 'lxml')
print(content)

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="/static/shared-web/favicon-4e1897dfd0901e8a3bf7e604d3a90636.ico" rel="icon"/>
<link href="/static/shared-web/iphone-7c4d7dcb05fece466d8901945e36bbaa.png" rel="apple-touch-icon"/>
<link href="/static/shared-web/ipad-96988dd1c0902fd20b34ce260d03729b.png" rel="apple-touch-icon" sizes="76x76"/>
<link href="/static/shared-web/iphoneRetina-c772d091f011ef7ce26f631360de4907.png" rel="apple-touch-icon" sizes="120x120"/>
<link href="/static/shared-web/ipadRetina-8a06978f3bf7985f413edcb5113c9783.png" rel="apple-touch-icon" sizes="152x152"/>
<meta content="https://www.jobstreet.com.sg/static/shared-web/banner-0c2ac79883746c7700892a4915e53610.png" name="twitter:image"/>
<meta content="summary" name="twitter:card"/>
<meta content="@JobStreetSG" name="twitter:site"/>
<meta content="https://www.jobstreet.com.sg/static/shared-web/banner-0c2ac79883746c7700892a

In [19]:
def scrape_job_details(url):
    service = Service(executable_path="/chromedriver")
    driver = webdriver.Chrome(service=service)

    driver.get(url)
    content = driver.find_elements(By.CSS_SELECTOR,'.z1s6m00._1hbhsw67i._1hbhsw66e._1hbhsw69q._1hbhsw68m._1hbhsw6n._1hbhsw65a._1hbhsw6ga._1hbhsw6fy')    
    jobs_list = []
    for post in content:
        data = {
                  "job_title": None,
                  "company": None,
                  "Location": None,
                  "Salary": None,
            }
        try:
            data["job_title"] = post.find_element(By.CLASS_NAME,'z1s6m00').text
        except:
            pass
        try:
            data["company"] = post.find_element(By.CSS_SELECTOR, 'div:nth-child(1) > div.z1s6m00._1hbhsw6ce > span > span').text
        except:
            pass
        try:
            data["Location"] = post.find_element(By.CSS_SELECTOR, '.z1s6m00._1hbhsw64y.y44q7i0.y44q7i3.y44q7i21.y44q7ih').text
        except:
            pass
        try:
            data["Salary"] = post.find_element(By.CSS_SELECTOR, 'div:nth-child(1) > span:nth-child(4)').text
        except:
            pass

        jobs_list.append(data)
    return pd.DataFrame(jobs_list)

In [20]:
df = scrape_job_details(current_url)
# df_jobs.to_csv("jobstreet.csv", index=False)  
df

,job_title,company,Location,Salary
0,Data Scientist (Partner Company)\nSGInnovate\n...,SGInnovate,Singapore,SGD 5K - 7K monthly
1,Senior Data Analyst\nCloudun Technology Pte. L...,Cloudun Technology Pte. Ltd.,Queenstown,None
2,Senior Data Scientist - Pricing & Performance ...,JobStreet,Downtown Core,None
3,Data Scientist\nUp Devlabs Pte. Ltd.\nWest\nSG...,Up Devlabs Pte. Ltd.,West,SGD 15K - 30K monthly
4,Data Scientist (Junior / Senior)\nComfort Tran...,Comfort Transportation Pte Ltd & CityCab Pte Ltd,North-East,None
5,Data Scientist (statistics)\nALLIED SEARCH PTE...,ALLIED SEARCH PTE. LTD.,Singapore,SGD 4.5K - 7.5K monthly
6,Data Analyst / Data Scientist (Office of Data ...,National University of Singapore,Singapore,None
7,Data Scientist\nOptimum Solutions (S) Pte Ltd\...,Optimum Solutions (S) Pte Ltd,East,SGD 9K - 11K monthly
8,Data Scientist [Contract/SQL]\nOptimum Solutio...,Optimum Solutions (S) Pte Ltd,Central,SGD 7K - 12K monthly
9,"Senior/ Analyst, Data Science, Group Finance A...",Singapore Health Services Pte Ltd (SingHealth HQ),Bukit Merah,None
